In [2]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd
from cloudy_fit_lib import *
from scipy.interpolate import RegularGridInterpolator
import pickle

In [3]:
rootdir = '/home/gayatri/Documents/cloudy_scripts/blr/grids/'

In [4]:
hm12_z_grid, hm12_wav_grid, hm12_J_nu_grid = read_uvb('', 'hm12_galaxy.ascii')

Fetch the SED at $z=0.45$.

In [9]:
z_test = 0.45
hm12_J_nu_test = fetch_sed(z_test, hm12_z_grid, hm12_J_nu_grid)

For a grid of N_HI values.

In [45]:
logN_HI_min_main = 12
logN_HI_max_main = 12.25
logN_HI_step_main = 0.25

logN_HI_arr_main = np.arange(logN_HI_min_main, logN_HI_max_main+logN_HI_step_main, logN_HI_step_main)
file_list_main = create_grid_file_list(logN_HI_arr_main)
file_list_main[0]

'igm_lalpha_hm12_grid_1200'

In [46]:
log_hdens_min_main = -4.25
log_hdens_max_main = 1
log_hdens_step_main = 0.25
log_hdens_arr_main = np.arange(log_hdens_min_main, log_hdens_max_main+log_hdens_step_main, log_hdens_step_main)

log_metals_min_main = -0.25
log_metals_max_main = 1
log_metals_step_main = 0.25
log_metals_arr_main = np.arange(log_metals_min_main, log_metals_max_main+log_metals_step_main, log_metals_step_main)

In [47]:
len(logN_HI_arr_main)*len(log_hdens_arr_main)*len(log_metals_arr_main)

264

Process the grid to identify "failures"

In [48]:
log_T_pie_thresh = 4

# Bifurcate failures based on stopping temperature
low_temp_failures = []
high_temp_failures = []

In [49]:
for i in range(len(logN_HI_arr_main)):
    logN_HI = logN_HI_arr_main[i]

    # The filename corresponding to the current stopping HI column density
    filename = file_list_main[i]

    # Get list of densities and metallicities for this stopping HI column density 
    log_hdens_grid, log_metals_grid = read_grd_file(rootdir, filename)

    # Get average (log) HI temperatures for all grid points
    log_temps_grid = read_avr_file(rootdir, filename)

    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)

    for j in range(len(log_hdens_arr_main)):
        
        log_hdens = log_hdens_arr_main[j]
        
        for k in range(len(log_metals_arr_main)):
            
            log_metals = log_metals_arr_main[k]
            
            # Get grid index number for the current n_H and metallicity
            idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
            
            # Isolate the average temperature and column density for all species
            log_temp = log_temps_grid[idx]
            log_col_dens = log_col_dens_grid[idx]
            
            # Check if this grid point is a failure
            if np.round(log_col_dens[0],2) != logN_HI:
                # Categorize based on temperature
                if log_temp<log_T_pie_thresh:
                    low_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])
                else:
                    high_temp_failures.append([logN_HI, log_hdens, log_metals, log_temp, log_col_dens[0]])

/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))


In [50]:
low_temp_failures = np.array(low_temp_failures)
high_temp_failures = np.array(high_temp_failures)

In [51]:
len(low_temp_failures), len(high_temp_failures)

(0, 0)

No failures, moving on.

In [58]:
species_logN_samples_final = {}
logT_grid_final = np.zeros((len(logN_HI_arr_main), len(log_hdens_arr_main), len(log_metals_arr_main)))

In [59]:
for i in range(len(logN_HI_arr_main)):
    
    logN_HI = logN_HI_arr_main[i]
    
    # The filename corresponding to the current stopping HI column density
    filename = file_list_main[i]
    
    # Get list of densities and metallicities for this stopping HI column density 
    log_hdens_grid, log_metals_grid = read_grd_file(rootdir, filename)
        
    # Get average (log) HI temperatures for all grid points
    log_temps_grid = read_avr_file(rootdir, filename)
    
    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)
    
    for j in range(len(log_hdens_arr_main)):
        
        log_hdens = log_hdens_arr_main[j]
        
        for k in range(len(log_metals_arr_main)):
            
            log_metals = log_metals_arr_main[k]
            
            # Get grid index number for the current n_H and metallicity
            idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
            # Isolate the average temperature and column density for all species
            log_temp = log_temps_grid[idx]
            logT_grid_final[i,j,k] = log_temp
            log_col_dens = log_col_dens_grid[idx]
            
            # For each species
            for l in range(len(species_names)):
                
                s = species_names[l]
                
                if s not in species_logN_samples_final.keys():
                    
                    species_logN_samples_final[s] = -99.*np.ones((len(logN_HI_arr_main), 
                                                            len(log_hdens_arr_main),
                                                            len(log_metals_arr_main)))
                
                # Check for converged logN(HI)
                if np.round(log_col_dens[0],2) == logN_HI:
                    species_logN_samples_final[s][i,j,k] = log_col_dens[l]

/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))


In [60]:
unique, counts = np.unique(species_logN_samples_final['#column density H'].flatten(), return_counts=True)

In [61]:
counts

array([132, 132])

Load the subgrid.

In [62]:
logN_HI_min_sub = 12
logN_HI_max_sub = 12.25
logN_HI_step_sub = 0.25

logN_HI_arr_sub = np.arange(logN_HI_min_sub, logN_HI_max_sub+logN_HI_step_sub, logN_HI_step_sub)

file_list_sub = create_grid_file_list(logN_HI_arr_sub)

log_hdens_min_sub = -4.25
log_hdens_max_sub = 1
log_hdens_step_sub = 0.25

log_hdens_arr_sub = np.arange(log_hdens_min_sub, log_hdens_max_sub+log_hdens_step_sub, log_hdens_step_sub)

log_metals_min_sub = -0.25
log_metals_max_sub = 1
log_metals_step_sub = 0.25

log_metals_arr_sub = np.arange(log_metals_min_sub, log_metals_max_sub+log_metals_step_sub, log_metals_step_sub)

# Total number of grid points
len(logN_HI_arr_sub)*len(log_hdens_arr_sub)*len(log_metals_arr_sub)

264

In [63]:
species_logN_samples_sub = {}
logT_grid_sub = np.zeros((len(logN_HI_arr_sub), len(log_hdens_arr_sub), len(log_metals_arr_sub)))

In [64]:
for i in range(len(logN_HI_arr_sub)):
    
    logN_HI = logN_HI_arr_sub[i]
    
    # The filename corresponding to the current stopping HI column density
    filename = file_list_sub[i]
    
    # Get list of densities and metallicities for this stopping HI column density 
    log_hdens_grid, log_metals_grid = read_grd_file(rootdir, filename)
    
    # Get average (log) HI temperatures for all grid points
    log_temps_grid = read_avr_file(rootdir, filename)
    
    # Get column densities for all species
    species_names, log_col_dens_grid = read_col_file(rootdir, filename)
    
    for j in range(len(log_hdens_arr_sub)):
        
        log_hdens = log_hdens_arr_sub[j]
        
        for k in range(len(log_metals_arr_sub)):
            
            log_metals = log_metals_arr_sub[k]
            
            # Get grid index number for the current n_H and metallicity
            idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
            
            # Isolate the average temperature and column density for all species
            log_temp = log_temps_grid[idx]
            logT_grid_sub[i,j,k] = log_temp
            
            log_col_dens = log_col_dens_grid[idx]
            
            # For each species
            for l in range(len(species_names)):
                
                s = species_names[l]
                
                if s not in species_logN_samples_sub.keys():
                    
                    species_logN_samples_sub[s] = -99.*np.ones((len(logN_HI_arr_sub), 
                                                            len(log_hdens_arr_sub),
                                                            len(log_metals_arr_sub)))
                
                # Check for converged logN(HI)
                if np.round(log_col_dens[0],2) == logN_HI:
                    species_logN_samples_sub[s][i,j,k] = log_col_dens[l]

/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))
/home/gayatri/Documents/cloudy_scripts/blr/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))


In [65]:
# Consider a low temperature failure
for f in range(len(low_temp_failures)):
    # Register the "coordinates" of the failed grid point
    logN_HI = low_temp_failures[f][0]
    log_hdens = low_temp_failures[f][1]
    log_metals = low_temp_failures[f][2]

    # Get indices for the grid point in the main grid
    idx_logN_HI_main = np.where(logN_HI_arr_main==logN_HI)[0][0]
    idx_log_hdens_main = np.where(log_hdens_arr_main==log_hdens)[0][0]
    idx_log_metals_main = np.where(log_metals_arr_main==log_metals)[0][0]

    # Get indices for the grid point in the subgrid
    idx_logN_HI_sub = np.where(logN_HI_arr_sub==logN_HI)[0][0]
    idx_log_hdens_sub = np.where(log_hdens_arr_sub==log_hdens)[0][0]
    idx_log_metals_sub = np.where(log_metals_arr_sub==log_metals)[0][0]
    
    # For each species
    for l in range(len(species_logN_samples_final)):

        s = list(species_logN_samples_final.keys())[l]
        # Set the temperature column density for the species in the final grid to that taken from the subgrid
        logT_grid_final[idx_logN_HI_main,idx_log_hdens_main,idx_log_metals_main] = logT_grid_sub[idx_logN_HI_sub,idx_log_hdens_sub,idx_log_metals_sub]
        species_logN_samples_final[s][idx_logN_HI_main, idx_log_hdens_main, idx_log_metals_main] = species_logN_samples_sub[s][idx_logN_HI_sub, idx_log_hdens_sub, idx_log_metals_sub]

In [66]:
unique, counts = np.unique(species_logN_samples_final['#column density H'].flatten(), return_counts=True)
counts

array([132, 132])

In [67]:
grid_points_ext = np.array([[16.  , -4.25,  1.  ],
       [16.25, -5.  ,  1.  ],
       [16.25, -4.75,  1.  ],
       [16.25, -4.5 ,  1.  ],
       [16.25, -4.25,  1.  ],
       [16.5 , -5.  ,  0.75],
       [16.5 , -5.  ,  1.  ],
       [16.5 , -4.75,  0.75],
       [16.5 , -4.75,  1.  ],
       [16.5 , -4.5 ,  1.  ],
       [16.5 , -4.25,  1.  ],
       [16.75, -5.  ,  0.75],
       [16.75, -5.  ,  1.  ],
       [16.75, -4.75,  0.75],
       [16.75, -4.75,  1.  ],
       [16.75, -4.5 ,  0.75],
       [16.75, -4.5 ,  1.  ],
       [16.75, -4.25,  1.  ],
       [17.  , -5.  ,  0.75],
       [17.  , -5.  ,  1.  ],
       [17.  , -4.75,  0.75],
       [17.  , -4.75,  1.  ],
       [17.  , -4.5 ,  0.75],
       [17.  , -4.5 ,  1.  ],
       [17.  , -4.25,  1.  ]])

len(grid_points_ext)

25

In [68]:
# For each species
for l in range(len(species_logN_samples_final)):
    
    s = list(species_logN_samples_final.keys())[l]
    
    # Consider a high temperature failure
    # These exactly grid_points_ext (excepting any extra column)
    for f in range(len(high_temp_failures)):
        
        # Register the "coordinates" of the failed grid point
        logN_HI = high_temp_failures[f][0]
        log_hdens = high_temp_failures[f][1]
        log_metals = high_temp_failures[f][2]
        
        # Get indices for the grid point in the main grid
        idx_logN_HI_main = np.where(logN_HI_arr_main==logN_HI)[0]
        idx_log_hdens_main = np.where(log_hdens_arr_main==log_hdens)[0]
        idx_log_metals_main = np.where(log_metals_arr_main==log_metals)[0]
        
        # Set the temperature column density for the species in the final grid to that taken from the subgrid
        logT_grid_final[idx_logN_HI_main, idx_log_hdens_main, idx_log_metals_main] = logT_ext_grid[f]
        species_logN_samples_final[s][idx_logN_HI_main, idx_log_hdens_main, idx_log_metals_main] = species_logN_samples_ext[s][f]


In [69]:
np.unique(np.round(species_logN_samples_final['#column density H'], 2)) == logN_HI_arr_main

array([ True,  True])

In [70]:
output = open(rootdir+'final_grid.pkl', 'wb')
pickle.dump(species_logN_samples_final, output)
output.close()

In [71]:
np.savetxt(rootdir+'final_flat_logT.dat', logT_grid_final.flatten())


In [72]:
species_fracs_samples_final = {s:np.zeros((len(logN_HI_arr_main),len(log_hdens_arr_main),len(log_metals_arr_main))) for s in species_names_ions}

# H fractions
N_H_grid_final = 10**species_logN_samples_final['#column density H']+10**species_logN_samples_final['H+']
species_fracs_samples_final['#column density H'] = 10**species_logN_samples_final['#column density H']/N_H_grid_final
species_fracs_samples_final['H+'] = 10**species_logN_samples_final['H+']/N_H_grid_final

N_He_grid_final = 10**species_logN_samples_final['He']+10**species_logN_samples_final['He+']+10**species_logN_samples_final['He+2']
species_fracs_samples_final['He'] = 10**species_logN_samples_final['He']/N_He_grid_final
species_fracs_samples_final['He+'] = 10**species_logN_samples_final['He+']/N_He_grid_final
species_fracs_samples_final['He+2'] = 10**species_logN_samples_final['He+2']/N_He_grid_final

# Grid of metallicities

log_metals_grid_final = np.zeros((len(logN_HI_arr_main),len(log_hdens_arr_main),len(log_metals_arr_main)))

for i in range(len(logN_HI_arr_main)):
    for j in range(len(log_hdens_arr_main)):
        log_metals_grid_final[i,j,:] = log_metals_arr_main
        
for i in range(5,len(species_names_ions)):
    s = species_names_ions[i]
    species_fracs_samples_final[s] = 10**species_logN_samples_final[s]/(N_H_grid_final*10**log_metals_grid_final)

species_logf_samples_final = {s:np.log10(species_fracs_samples_final[s]) for s in list(species_fracs_samples_final.keys())}

output = open(rootdir+'final_grid_logf.pkl', 'wb')
pickle.dump(species_logf_samples_final, output)
output.close()

/tmp/ipykernel_66761/4135434002.py:25: RuntimeWarning: divide by zero encountered in log10
  species_logf_samples_final = {s:np.log10(species_fracs_samples_final[s]) for s in list(species_fracs_samples_final.keys())}
